run luenberger estimator

In [ ]:
import sys, os
import numpy as np

sys.path.append('../')

import c4dynamics as c4d

import m_target

import m_radar
import m_filtertype

# initial conditions 
z0 = 100000 * c4d.ft2m
v0 = -6000 * c4d.ft2m
beta = 500 * c4d.lbft2tokgm2

rdr = c_radar(np.array([z0 + 25 * c4d.ft2m, v0 - 150 * c4d.ft2m, beta + 300 * c4d.lbft2tokgm2])
              , m_filtertype.filtertype.luenberger)


print('is observable:')
print(rdr.filter.isobservable())
print('desired eigen values:')
print(rdr.filter.eig())
print('desired eigen values:')
print(np.array([-4, -4, -4]))

rdr.filter.setest(np.array([-4, -4, -4]))




In [ ]:
def nonlinear(x, t):
    dx = np.zeros(len(x))
    dx[0] = x[1]
    dx[1] = .0034 * np.exp(-x[0] / 22000) * x[1]**2 / 2 / x[2] - c4d.g
    dx[2] = 0
    return dx

def nonlinear_simp(x, t):
    dx = np.zeros(len(x))
    dx[0] = x[1]
    dx[1] = .0034 * x[1]**2 / 2 / x[2] - c4d.g
    dx[2] = 0
    return dx

def linear(x, t, b0): # linearized wrt the simplified nonlinear system. namely where the density is constant over all the altitudes. 
    dx = np.zeros(len(x))
    dx[0] = x[1]
    dx[1] = -np.sqrt(2 * 0.0034 * c4d.g / b0) * x[1] - c4d.g / b0 * x[2]
    dx[2] = 0
    return dx


In [23]:
from scipy.integrate import solve_ivp, odeint

# initial conditions 
z0 = 100000 * c4d.ft2m
v0 = -6000 * c4d.ft2m
a0 = 0
beta = 500 * c4d.lbft2tokgm2

# sim parameters 
dt = 1e-3
tf = 30 
t = np.arange(0, tf, dt)

  
''' simulate nonlinear system '''
y_nln = odeint(nonlinear, [z0, v0, beta], t) 
y_nlnsimp = odeint(nonlinear_simp, [z0, v0, beta], t) 
y_lin = odeint(linear, [z0, v0, beta], t, args = (beta, )) 


In [ ]:
plt.plot(t, y_nlnsimp[:, 0], label = 'simplified nonlinear')
plt.plot(t, y_lin[:, 0], label = 'linear')
plt.legend()
plt.title('Altitude')
plt.show()

plt.plot(t, y_nlnsimp[:, 1], label = 'simplified nonlinear')
plt.plot(t, y_lin[:, 1], label = 'linear')
plt.legend()
plt.title('Velocity')
plt.show()

plt.plot(t, y_nlnsimp[:, 2], label = 'simplified nonlinear')
plt.plot(t, y_lin[:, 2], label = 'linear')
plt.legend()
plt.title(r'$\beta$')
plt.show()


In [ ]:
plt.plot(t, y_nln[:, 0], label = 'nonlinear')
plt.plot(t, y_lin[:, 0], label = 'linear')
plt.legend()
plt.title('Altitude')
plt.show()

plt.plot(t, y_nln[:, 1], label = 'nonlinear')
plt.plot(t, y_lin[:, 1], label = 'linear')
plt.legend()
plt.title('Velocity')
plt.show()

plt.plot(t, y_nln[:, 2], label = 'nonlinear')
plt.plot(t, y_lin[:, 2], label = 'linear')
plt.legend()
plt.title(r'$\beta$')
plt.show()


In [ ]:
from scipy.integrate import solve_ivp, odeint



# initial conditions 
z0 = 100000 * c4d.ft2m
v0 = -6000 * c4d.ft2m
a0 = 0
beta = 500 * c4d.lbft2tokgm2

# sim parameters 
dt = 1e-3
tf = 30 

# objects definition 
tgt = target([z0, v0, a0], beta)
rdr = radar([z0 + 25 * c4d.ft2m, v0 - 150 * c4d.ft2m, beta + 300 * c4d.lbft2tokgm2])

# main loop 
for t in np.arange(0, tf, dt):
    
    ''' simulate target motion '''
    y = odeint(target.deriv, [tgt.z, tgt.vz], t + [0, dt], args = (tgt, )) 
    # if y[-1, 0] <= 0:
    #     break
    tgt.z, tgt.vz = y[-1, :]
    tgt.data = np.concatenate((tgt.data, np.array([[tgt.z, tgt.vz, 0, 0]])), axis = 0)
    
    ''' take radar measure '''
    tgt_dist = tgt.z + rdr.vk * np.random.randn(1) #  0.86217 # 
    tgt_measure = rdr.measure(t, tgt_dist)

# zmask = tgt.data[:, 0] < 100000


dy = target.deriv(tgt.data, 0, tgt) 
tgt.data[:, 2] = np.asarray(dy).T[:, 0]
tgt.data[:, 3] = np.ones(len(tgt.data)) * beta